# 拓扑排序介绍

## 基本概念

在图论中，拓扑排序（Topological Sorting）是一个有向无环图（DAG, Directed Acyclic Graph）的所有顶点的线性序列。  
且该序列必须满足下面两个条件：

1. 每个顶点出现且只出现一次。
2. 若存在一条从顶点 A 到顶点 B 的路径，那么在序列中顶点 A 出现在顶点 B 的前面。

注：有向无环图（DAG）才有拓扑排序，非DAG图没有拓扑排序一说。

> 有向无环图的定义为：如果一个有向图从任意顶点出发无法经过若干条边回到这个点，则称这个图是一个有向无环图。

# Kahn's 算法

该方法每次从拓扑图中删去入度为0的顶点，并删除与其相关的边，直到无入度为0的顶点存在。由于拓扑图的环路中不存在入度为零的点，因此该方法可以自动识别有环图。为了快速的计算每个顶点的入度，算法需要一个辅助数据来记录拓扑图中每个顶点的入度。  
算法的具体实现流程如下：

1. 从DAG图中选择一个没有前驱（即入度为0）的顶点并输出。
2. 从图中删除该顶点和所有以它为起点的有向边。
3. 重复步骤1和2直到当前的DAG图为空或当前图中不存在无前驱的顶点为止。

**时间复杂度**： **$O(n)$** 遍历拓扑图中的每个顶点， 其中 $n$ 代表拓扑图中顶点的数目。

**空间复杂度**： **$O(n)$** 需要辅助数据记录每个顶点的入度，以及堆栈来临时保存入度为0的顶点。

In [ ]:
# Kahn's algorithm实现
import algviz

def topologicalSortingKahns(graph):
    # 计算每个顶点的入度。
    viz = algviz.Visualizer(3.0)
    graph_viz = viz.createGraph(data=list(graph.values()), horizontal=True)
    degree = viz.createVector(data=[0 for _ in range(len(graph))], name='NodeDegree')
    for u in graph.values():
        for v in u.neighbors():
            degree[v[0].val] += 1
            viz.display(1.0)
    que = viz.createVector(name='Stack')
    # 将入度为0的点放入队列。
    for u in graph.values():
        if degree[u.val] == 0:
            que.append(u)
        viz.display(1.0)
    # 取点，删边，加点。
    res = viz.createVector(name='Result')
    while len(que) > 0:
        cur_node = que.pop()
        for neighbor in cur_node.neighbors():
            v = neighbor[0]
            degree[v.val] -= 1
            if degree[v.val] == 0:
                que.append(v)
        res.append(cur_node.val)
        viz.display()
    return res._data

graph1 = algviz.parseGraph([[0,1], [0,3], [0,5], [1,2], [3,2], [3,4], [2,4], [5,3]])
print("Call Kahn's topological sorting algorithm:")
print("Result:", topologicalSortingKahns(graph1))

# 深度优先搜索拓扑排序：

在使用深度优先搜索(DFS)的方式遍历拓扑图时，可以按照一定的顺序将图中的顶点打印出来，为了实现打印出来顶点的拓扑排序，需要在一个顶点的邻居点之前打印它。  

时间复杂度： **$O(n)$** 需要遍历图中的每个顶点， $n$ 代表顶点数目。

空间复杂度： **$O(n)$** 递归的堆栈深度和图中最长路径呈正比，此外还需要记录每个顶点是否已被访问过。

下面的代码实现了基于深度优先搜索的拓扑排序：

In [ ]:
# 基于深度优先搜索的拓扑排序算法实现（递归版本）。
class Solution:
    def topologicalSortingDfsUnit(self, node, visited, stack):
        visited[node.val] = True
        for v in node.neighbors():
            if not visited[v[0].val]:
                self.topologicalSortingDfsUnit(v[0], visited, stack)
        stack.insert(0, node.val)
        self.viz.display()

    def topologicalSortingDfs(self, graph):
        self.viz = algviz.Visualizer()
        graph_viz = self.viz.createGraph(data=list(graph.values()), horizontal=True)
        visited = [False for _ in range(len(graph))]
        stack = self.viz.createVector(name='Result')
        for u in graph.values():
            if not visited[u.val]:
                self.topologicalSortingDfsUnit(u, visited, stack)
        return stack._data

print('Call DFS topological sorting algorithm:')
print('Result:', Solution().topologicalSortingDfs(graph1))

# 传统的深度优先搜索

基于DFS的拓扑排序算法与传统的DFS方法极为相似，两者之间的唯一不同之处在于：**DFS拓扑排序是先递归后访问节点，而传统的DFS是先访问节点后递归。**  
下面的代码实现了传统的DFS算法，对比可以发现两者的具体区别：

In [ ]:
# 传统的拓扑图的深度优先搜索算法（递归版本）。
class DFS:
    def graphDfsUnit(self, node, visited, stack):
        visited[node.val] = True
        stack.append(node.val)
        self.viz.display()
        for v in node.neighbors():
            if not visited[v[0].val]:
                self.graphDfsUnit(v[0], visited, stack)

    def graphDfs(self, graph):
        self.viz = algviz.Visualizer()
        graph_viz = self.viz.createGraph(data=list(graph.values()), horizontal=True)
        visited = [False for _ in range(len(graph))]
        stack = self.viz.createVector(name='TraverseNodes')
        for u in graph.values():
            if not visited[u.val]:
                self.graphDfsUnit(u, visited, stack)
        return stack._data

print('Call DFS traverse algorithm:')
print('Return:', DFS().graphDfs(graph1))

# 有环图的处理

前面提到，Kanh's算法可以检测到有环图的存在，而基于DFS的拓扑排序算法则无法识别有环图，进而会错误的对环中的节点进行排序。 

下面的测试通过一个简单的有环图对比了两种算法的运行行为：  
通过观察可以发现，Kahn's 算法只会对图中无环部分的节点进行排序，而基于DFS的拓扑排序算法会对整个图进行排序，尽管子环路中的节点不应该被放到结果中。（*但是Stackoverflow中有位大神说通过改造可以解决DFS拓扑排序算法无法识别子环路的问题，有时间研究一下。*）

In [ ]:
# 有环图的处理。
graph2 = algviz.parseGraph([[0,1], [1,2], [2,3], [3,1]])
print("Call Kahn's topological sorting algorithm:")
print("Result:", topologicalSortingKahns(graph2))
print('-'*50)
print('Call DFS topological sorting algorithm:')
print('Result:', Solution().topologicalSortingDfs(graph2))

# 参考链接

+ https://zhuanlan.zhihu.com/p/56024487
+ https://songlee24.github.io/2015/05/07/topological-sorting/
+ https://www.geeksforgeeks.org/topological-sorting/